<a href="https://colab.research.google.com/github/Harsh-L/AttendanceSystem/blob/main/Sem_6_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Recognition Based Attendance System




# Installing Libraries And Mounting Google Drive

In [ ]:
!pip install streamlit
!pip install pyngrok
#!pip install streamlit_ace
!pip install face_recognition
from google.colab import drive
drive.mount('/content/drive')
print("Libraries Installed")

# Registering A Person For Face Recognition And Storing Data

In [ ]:
# %%writefile Register.py
from csv import writer
import pandas as pd
import numpy as np
import streamlit as st
import cv2
import os
import re

data = pd.DataFrame(columns=['ID', 'First Name', 'Middle Name', 'Last Name', 'Gender', 'Department'])
video_path = '/content/drive/MyDrive/Data Science/Face Recognition/Videos'
person_info_path = '/content/drive/MyDrive/Data Science/Face Recognition/Person Data/person_information.csv'
department_info_path = '/content/drive/MyDrive/Data Science/Face Recognition/Department Data/department_information.csv'


st.title("Person's Detail")
process = st.radio("",["Insert", "Update", "Delete"])

if process == "Insert":
  try :
    dept_data_df = pd.read_csv(department_info_path)
  except : # Catching Exception
    st.error("Department Data Is Empty")
  else :
    usr_id = st.number_input("ID    (1-150) ", 1000000000,9999999999)
    usr_fname = st.text_input("First Name  (3-25)" , max_chars=25)
    usr_mname = st.text_input("Middle Name  (3-25)" , max_chars=25)
    usr_lname = st.text_input("Last Name  (3-25)" , max_chars=25)
    usr_gender = st.radio("Gender", ["Male", "Female"])
    usr_dept = st.selectbox("Department", dept_data_df['Name'], index=1)
    usr_video = st.file_uploader("Upload Person's Video", type=['mp4'])
    submit = st.button("Submit")

    if submit:
      # Validation Data Fields
      if usr_id <= 9999999999 and usr_id >=1000000000 and len(usr_fname) > 2 and len(usr_mname) > 2 and len(usr_lname) > 2 and usr_gender != None  and usr_dept != None and usr_video != None:
        # Finding Department ID
        usr_dept_id = np.where( ( (dept_data_df['Name'] == usr_dept) ), dept_data_df['ID'], None)

        # Converting Person Data To Numpy Array And Storing It As Pandas Dataframe
        temp_data = pd.DataFrame(np.insert(data.values, len(data.index), values=[usr_id, usr_fname, usr_mname, usr_lname, usr_gender, usr_dept_id], axis=0))
        temp_data.columns = data.columns    # Copying Columns
        data = data.append(temp_data)   # Appending Temporary Dataframe To Pre Defined Dataframe
        # Checking If Orignial Dataframe Is Empty Or Not
        if person_data_df.empty :    # If Empty Include Header In Orignal Dataframe i.e. Column Names
          data.to_csv(person_info_path, mode='a', index=False, header=True)
        else :
          data.to_csv(person_info_path, mode='a', index=False, header=False)

        st.success("Data Entered Successfully")
        # Defining Naming Scheme For Videos Uploaded
        usr_video_name = str(usr_id)
        # Creating A Video File
        usr_video_path = os.path.join("/content/drive/MyDrive/Data Science/Face Recognition/Videos", usr_video_name)
        # Saving/Writting Video To Named File
        with open(os.path.join("/content/drive/MyDrive/Data Science/Face Recognition/Videos", usr_video_name), "wb") as f:
          f.write(usr_video.getbuffer())

        # Getting Video From Folder To Get Images
        cap = cv2.VideoCapture(video_path +"/"+ usr_video_name)
        frame = 15
        img_count = 0
        while True :
          success, image = cap.read() # Read Frame One By One Named Image
          if image is not None : # If image is There
            if frame % 15 == 0 and img_count <= 10 : # Take Every 15th Frame And If Count Of Image Is Less Than 10
              img_count = img_count + 1
              if(img_count < 10): # If Image Count is Less Than 10 Then Add 0 As A Prefix In Image Count
                save_img_name = usr_video_name + "_" + "0" + str(img_count) + ".jpg"
                cv2.imwrite('/content/drive/MyDrive/Data Science/Face Recognition/Images/'+save_img_name , image) # Writting Image In Folder
              else :
                save_img_name = usr_video_name + "_" + str(img_count) + ".jpg"
                cv2.imwrite('/content/drive/MyDrive/Data Science/Face Recognition/Images/'+save_img_name , image)
            frame = frame + 1 # Increasing Frame Counter
          else :
            break
        st.write("Video Uploaded Successfully")
        st.datarame(person_data_df)
      else :
        error_str = ""
        if len(usr_fname) < 25 and len(usr_fname) > 2 :
          error_str = error_str + "First Name Length Should Be (3-25) Characters " + "\n"
        if not bool(re.findall('[a-zA-Z]+', usr_fname)) :
          error_str = error_str + "First Name Should Contain Only Alphabets " + "\n"
        if len(usr_mname) < 25 and len(usr_mname) > 2 :
          error_str = error_str + "Middle Name Length Should Be (3-25) Characters " + "\n"
        if not bool(re.findall('[a-zA-Z]+', usr_mname)) :
          error_str = error_str + "Middle Name Should Contain Only Alphabets " + "\n"
        if len(usr_lname) < 25 and len(usr_lname) > 2 :
          error_str = error_str + "Length Should Be (3-25) Characters " + "\n"
        if not bool(re.findall('[a-zA-Z]+', usr_lname)) :
          error_str = error_str + "Last Name Should Contain Only Alphabets " + "\n"
        if usr_gender is None:
          error_str = error_str + "Gender Is Not Selected " + "\n"
        if usr_dept is None:
          error_str = error_str + "Department Is Not Selected " + "\n"
        if usr_video is None:
          error_str = error_str + "Video File Is Empty " + "\n"
        st.error(error_str)

if process == "Update":
  try :
    person_data_df = pd.read_csv(person_info_path)
  except : # Catching Exception
    st.error("Person Data Is Empty")
  else : # If Exception Not Occurs
    try :
      dept_data_df = pd.read_csv(department_info_path)
    except :
      st.error("Department Data is Empty")
    else :
      re_video = None
      selected_id = st.selectbox("ID", person_data_df.ID, index=0)

      index = person_data_df[ person_data_df['ID'] == selected_id ].index
      usr_fname = person_data_df.loc[index]["First Name"]
      usr_mname = person_data_df.loc[index]["Middle Name"]
      usr_lname = person_data_df.loc[index]["Last Name"]
      usr_gender = person_data_df.loc[index]["Gender"]
      #usr_dept_id = person_data_df.loc[index]["Department"]
      #dept_index = dept_data_df[ dept_data_df['ID'] == usr_dept_id ].index
      #usr_dept = dept_data_df[dept_index]['Name']

      #usr_dept = np.where( ( (dept_data_df['ID'] == usr_dept_id) ), dept_data_df['Name'], None)

      usr_fname = st.text_input("First Name  (3-25)" , max_chars=25, value = str(usr_fname[0]))
      usr_mname = st.text_input("Middle Name  (3-25)" , max_chars=25, value = str(usr_mname[0]))
      usr_lname = st.text_input("Last Name  (3-25)" , max_chars=25, value = str(usr_lname[0]))
      if str(usr_gender[0]) == "Male" :
        gen_index = 0
      elif str(usr_gender[0]) == "Female" :
        gen_index = 1
      usr_gender = st.radio("Gender", ["Male", "Female"], index=gen_index)
      usr_dept = st.selectbox("Department", dept_data_df.Name, index=dept_index)
      re_video = st.checkbox("Re-Upload Video")
      if re_video:
        usr_video = st.file_uploader("Upload Person's Video", type=['mp4'])
      st.write(selected_id)




      update = st.button("Update")

      if update:
        # Validation Data Fields
        if len(usr_fname) > 2 and len(usr_mname) > 2 and len(usr_lname) > 2 and usr_gender != None and usr_dept != None :
          # Finding Department ID
          #usr_dept_id = np.where( ( (dept_data_df['Name'] == usr_dept) ), dept_data_df['ID'], None)

          usr_dept_id_index = dept_data_df[ dept_data_df['Name'] == usr_dept ].index
          usr_dept_id = dept_data_df.loc[usr_dept_id_index, "ID"]

          index = person_data_df[ person_data_df['ID'] == selected_id ].index
          person_data_df.loc[index, "First Name"] = usr_fname
          person_data_df.loc[index, "Middle Name"] = usr_mname
          person_data_df.loc[index, "Last Name"] = usr_lname
          person_data_df.loc[index, "Gender"] = usr_gender
          person_data_df.loc[index, "Department"] = usr_dept_id

          person_data_df.to_csv(person_info_path, mode='w', index=False, header=True)
          st.write(pd.DataFrame(person_data_df))
          st.success("Data Updated Successfully")

          if re_video and usr_video is not None:
            # Defining Naming Scheme For Videos Uploaded
            usr_video_name = str(usr_id)
            '''# Creating A Video File
            usr_video_path = os.path.join("/content/drive/MyDrive/Data Science/Face Recognition/Videos", usr_video_name)
            # Saving/Writting Video To Named File
            with open(os.path.join("/content/drive/MyDrive/Data Science/Face Recognition/Videos", usr_video_name), "wb") as f:
              f.write(usr_video.getbuffer())

            # Getting Video From Folder To Get Images
            cap = cv2.VideoCapture(video_path +"/"+ usr_video_name)
            frame = 15
            img_count = 0
            while True :
              success, image = cap.read() # Read Frame One By One Named Image
              if image is not None : # If image is There
                if frame % 15 == 0 and img_count < 10 : # Take Every 15th Frame And If Count Of Image Is Less Than 10
                  img_count = img_count + 1
                  if(img_count < 10): # If Image Count is Less Than 10 Then Add 0 As A Prefix In Image Count
                    save_img_name = usr_video_name + "_" + "0" + str(img_count) + ".jpg"
                    cv2.imwrite('/content/drive/MyDrive/Data Science/Face Recognition/Images/'+save_img_name , image) # Writting Image In Folder
                  else :
                    save_img_name = usr_video_name + "_" + str(img_count) + ".jpg"
                    cv2.imwrite('/content/drive/MyDrive/Data Science/Face Recognition/Images/'+save_img_name , image)
                frame = frame + 1 # Increasing Frame Counter
              else :
                break
            st.write("Video Uploaded Successfully")'''
        else :
          error_str = ""
          if len(usr_fname) < 25 and len(usr_fname) > 2 :
            error_str = error_str + "First Name Length Should Be (3-25) Characters " + "\n"
          if not bool(re.findall('[a-zA-Z]+', usr_fname)) :
            error_str = error_str + "First Name Should Contain Only Alphabets " + "\n"
          if len(usr_mname) < 25 and len(usr_mname) > 2 :
            error_str = error_str + "Middle Name Length Should Be (3-25) Characters " + "\n"
          if not bool(re.findall('[a-zA-Z]+', usr_mname)) :
            error_str = error_str + "Middle Name Should Contain Only Alphabets " + "\n"
          if len(usr_lname) < 25 and len(usr_lname) > 2 :
            error_str = error_str + "Length Should Be (3-25) Characters " + "\n"
          if not bool(re.findall('[a-zA-Z]+', usr_lname)) :
            error_str = error_str + "Last Name Should Contain Only Alphabets " + "\n"
          if usr_gender is None:
            error_str = error_str + "Gender Is Not Selected " + "\n"
          if usr_dept is None:
            error_str = error_str + "Department Is Not Selected " + "\n"
          if re_video is not None and usr_video is None:
            error_str = error_str + "Video File Is Empty " + "\n"
          st.error(error_str)

if process == "Delete":
  try:
    person_data_df = pd.read_csv(person_info_path)
  except:
    st.error("Person Data Is Empty")
  else:
    selected_id = st.selectbox("ID", person_data_df.ID, index=0)
    delete = st.button("Delete")
    selected_id_df = person_data_df.query("ID == @selected_id", inplace=False)
    if not selected_id_df.empty:
      st.dataframe(selected_id_df)

    if delete:
      new_person_data_df = person_data_df.drop(person_data_df.query("ID == @selected_id").index)
      new_person_data_df.to_csv(person_info_path, mode='w', index=False, header=True)
      if person_data_df.query("ID == @selected_id").empty :
        st.success("Data Deleted")
        st.dataframe(person_data_df)
      else:
        st.error("Error Occured When Deleting Data")


Writing Register.py


Running Website And Getting Authentication Token From Streamlit To Connect It To Python Code

In [ ]:
'''!streamlit run Register.py &>/dev/null&
!ngrok authtoken 26I9sBkkGEDt6JCFoxECzwgoOdM_3m86LJtsg6GyEpcdZVrVC'''
#!streamlit run Register.py &
from pyngrok import ngrok
!ngrok.kill()
!streamlit run Register.py &>/dev/null& # IF Above Fails Then Use This
!ngrok authtoken 26I9sBkkGEDt6JCFoxECzwgoOdM_3m86LJtsg6GyEpcdZVrVC

/bin/bash: -c: line 1: syntax error: unexpected end of file
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Getting The URL Of Website

In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect('8501')
public_url

<NgrokTunnel: "http://a514-35-231-149-61.ngrok.io" -> "http://localhost:8501">

In [ ]:
# TO Kill Streamlit Tunnel
#!streamlit run register.py
#ngrok.kill

# Recognising Face And Marking Attendance Of Person

In [ ]:
%%writefile Recognise.py
from csv import writer
from PIL import Image
import os
import cv2
import datetime
import face_recognition
import pandas as pd
import numpy as np
import streamlit as st

person_info_path = '/content/drive/MyDrive/Data Science/Face Recognition/Person Data/person_information.csv'
department_info_path = '/content/drive/MyDrive/Data Science/Face Recognition/Department Data/department_information.csv'
attendance_path = '/content/drive/MyDrive/Data Science/Face Recognition/attendance.csv'

person_info_df = pd.read_csv(person_info_path)
dept_info_df = pd.read_csv(department_info_path)
attendance_df = pd.read_csv(attendance_path)
attendance_index = pd.DataFrame(columns=['ID', 'Date', 'Entry Time', 'Exit Time', 'Department'])

# Function To Find The Encoded Data Of The Training Image
def findEncodings(images):
  encodeList = []
  for img in images:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    encode = face_recognition.face_encodings(img)[0]
    encodeList.append(encode)
  return encodeList

# Function To Recognise Person And Return ID And Name
def recognize_face(passed_image):
  path = '/content/drive/MyDrive/Data Science/Face Recognition/Images'
  images = []
  classNames = []
  #We define two empty lists for storing training images and the classNames, which means the person’s name.
  # Reading the training images and classes and storing into the corresponding lists
  for img in os.listdir(path):
    image = cv2.imread(f'{path}/{img}')
    images.append(image)
    classNames.append(os.path.splitext(img)[0])
  # Find Encodings Of Trained Images
  knownEncodes = findEncodings(images)
  #Note – Change this scale according to your need between 0 and 1.
  # A lower number will give better performance but it will not be able to detect faces if the face is small in the image,
  # and a greater number can detect small faces in the image but the performance will be slow
  scale = 0.25
  #box_multiplier = 1/scale
  #cap = cv2.VideoCapture(passed_image)
  #success, img = cap.read()
  #count = 0
  #while count < len(img):
    #if img is not None:
  #img = passed_image
  Current_image = cv2.resize(passed_image,(0,0),None,scale,scale)
  #Current_image = cv2.resize(img,(480,480),scale,scale)
  Current_image = cv2.cvtColor(Current_image, cv2.COLOR_BGR2RGB)
  # Find the face location and encodings for the current frame
  face_locations = face_recognition.face_locations(Current_image,  model='cnn')
  face_encodes = face_recognition.face_encodings(Current_image,face_locations)
  #count+=1

  for encodeFace,faceLocation in zip(face_encodes,face_locations):
    matches = face_recognition.compare_faces(knownEncodes,encodeFace, tolerance=0.6)
    faceDis = face_recognition.face_distance(knownEncodes,encodeFace)
    matchIndex = np.argmin(faceDis)
    # If match found then get the class name for the corresponding match
    if matches[matchIndex]:
      name = classNames[matchIndex].upper()
      return name
    else:
      name = 'Unknown'

  return name

# Function To Mark The Attendance Of The Person
def mark_attendance(passed_id) :

  person_info_df_modified = person_info_df.set_index("ID", inplace=False)
  person_dept =  person_info_df_modified.loc[passed_id]['Department']

  datetime_obj = datetime.datetime.now() # Creating Date Time Object
  current_date = datetime_obj.strftime("%d-%b-%Y") # Getting Current Date
  current_time = datetime_obj.strftime("%H:%M:%S") # Getting Current Time

  attendance_df_modified = attendance_df.set_index("ID", inplace=False)

  if attendance_df_modified.query("ID == @passed_id and Date == @current_date").empty:
    entry_time = current_time
    exit_time = None # None Will Convert Into NaN In Dataframe
    # Code for marking attendance Entry Time
    temp_data = pd.DataFrame(np.insert(attendance_index.values, len(attendance_index.index), values=[passed_id, current_date, entry_time, exit_time, person_dept], axis=0))
    temp_data.columns = attendance_index.columns    # Copying Columns
    attendance_index = attendance_index.append(temp_data)   # Appending Temporary Dataframe To Pre Defined Dataframe
    # Checking If Orignial Dataframe Is Empty Or Not
    if attendance_df.empty :    # If Dataframe Is Empty Include Header In Orignal Dataframe i.e. Column Names
      attendance_index.to_csv(attendance_path, mode='a', index=False, header=True)
    else :
      attendance_index.to_csv(attendance_path, mode='a', index=False, header=False)
  elif not attendance_df_modified.query("ID == @passed_id and Date == @current_date").empty:
    # Code for marking attendance Exit Time
    exit_time = current_time
    integer_passed_id = int(passed_id)
    attendance_df["Exit Time"] = np.where( ( (attendance_df['ID'] == passed_id) & (attendance_df['Date'] == current_date) ), exit_time, attendance_df["Exit Time"])
    attendance_df.to_csv(attendance_path, mode='w', index=False, header=True)
  else
    st.error("Error Occured Try Again")

  #person_info_df = pd.read_csv(person_info_path)
  #department_info_df = pd.read_csv(department_info_path)

  #department_info_df_modified = department_info_df.set_index("ID", inplace=False)
  #dept =  person_info_df_modified.loc[passed_id.to_string()]['Department']



#-------------------------------------------------------------------------------------------------------------------------------------#
st.title("Mark Your Attendence")
input_image = st.camera_input("Take Your Picture")

# USE WITH SPINNER STATUS ELEMENT TO REPRESENT LOADING WIDGET
with st.spinner():
  if input_image is not None:
    # Converting Streamlit Image To OpenCV Image
    image_byte_data = input_image.getvalue() # Getting The Byte Value Of Image
    cv_image = cv2.imdecode(np.frombuffer(image_byte_data, np.uint8), cv2.IMREAD_COLOR) # Converting Byte Image OpenCV Image
    cv_image_rgb = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB) # Converting Image To RGB Color Scheme

    person = recognize_face(cv_image_rgb) # Passing The Image To Function To Get ID And Name Of Person

    if person == "Unknown" or person is None: # If Person Is Not Recognized Then Display Error Message
      st.error("Error Person Not Found, Retry")
    else : # If Person Is Recognized Then Get ID And Name From The Lable Of Matched Image
      person_array = person.split("_")    # Splitting Lable To Get Array Of Elements Of Lable
      person_id = person_array[0]
      #person_first_name = person_array[1]
      #person_middle_name = person_array[2]
      #person_last_name = person_array[3]
      # Pass ID And Name To Mark The Attendance Of Recognised Person
      mark_attendance(person_id)
      # Display Success Message With Person ID And Name
      st.success("Attendance Marked Person : "+ person)


Writing Recognise.py


Running Website And Getting Authentication Token From Streamlit To Connect It To Python Code

In [ ]:
'''!streamlit run Recognise_And_Mark_Attendance.py &>/dev/null&
!ngrok authtoken 26I9sBkkGEDt6JCFoxECzwgoOdM_3m86LJtsg6GyEpcdZVrVC'''
#!streamlit run Recognise.py &
ngrok.kill()
!streamlit run Recognise.py &>/dev/null& # IF Above Fails Then Use This
!ngrok authtoken 26I9sBkkGEDt6JCFoxECzwgoOdM_3m86LJtsg6GyEpcdZVrVC

Getting The URL Of Website

In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect('8501')
public_url

# Viewing The Attendance

In [ ]:
%%writefile View.py

import pandas as pd
import numpy as np
import streamlit as st

person_info_path = '/content/drive/MyDrive/Data Science/Face Recognition/Person Data/person_information.csv'
department_info_path = '/content/drive/MyDrive/Data Science/Face Recognition/Department Data/department_information.csv'
attendance_path = '/content/drive/MyDrive/Data Science/Face Recognition/attendance.csv'

try :
  person_data_df = pd.read_csv(person_info_path)
except : # Catching Exception
  st.error("Person Data Is Empty")

try :
  dept_data_df = pd.read_csv(department_info_path)
except :
  st.error("Department Data Is Empty")
try :
  attendance_df = pd.read_csv(attendance_path)
except :
  st.error("Attendence Data Is Empty")


filtered_df = attendance_df.copy(deep=True) # True if Changes Should Not Reflect In Original DataFrame
filtered_df = filtered_df.merge(person_data_df,on='ID',how="left") # Merging / Left Join On Attendance Table And Person Table By ID
filtered_df.drop(['Gender', 'Department'], axis=1) # axis=1 As We Are Dropping Column
filtered_df = filtered_df[[ 'ID', 'First Name', 'Middle Name', 'Last Name', 'Date', 'Entry Time', 'Exit Time', 'Department' ]]

#filtered_df.insert(1, "First Name","",True) #Col Location, Col Name, Col Value, Repeat Values
#filtered_df.insert(2, "Middle Name","",True) #Col Location, Col Name, Col Value, Repeat Values
#filtered_df.insert(3, "Last Name","",True) #Col Location, Col Name, Col Value, Repeat Values
#filtered_df["First Name"] = np.where( ( (person_data_df["ID"] == filtered_df["ID"]) ), person_data_df["First Name"], None)
#filtered_df["Middle Name"] = np.where( ( (person_data_df["ID"] == filtered_df["ID"]) ), person_data_df["Middle Name"], None)
#filtered_df["Last Name"] = np.where( ( (person_data_df["ID"] == filtered_df["ID"]) ), person_data_df["Last Name"], None)
#filtered_df["First Name"] = person_data_df.loc[person_data_df["ID"] == filtered_df["ID"], "First Name"]

#person_data_df = person_data_df.set_index("ID", inplace=False) #If True, modifies the DataFrame in place (do not create a new object).

name_or_id = None
name_choice = None
id_choice = None
from_date = None
to_date = None
dept_choice = None
dept_id_choice = None

st.title("View Attendance")
by_id_name = st.checkbox("Filter By ID / Name")
by_date = st.checkbox("Filter By Date Interval")
by_department = st.checkbox("Filter By Department")
submit = st.button("Submit")

if by_id_name: #by_id_name return True if selected
  name_or_id = st.radio("Select Filter Criteria", ["ID", "Name"])
  if name_or_id == "ID":
    id_choice = st.selectbox("Pick ID", person_data_df.ID)
  elif name_or_id == "Name":
    # Combining FName, MName, LName And Creating New Column Full Name
    person_data_df['Full Name'] = person_data_df[['First Name', 'Middle Name', 'Last Name']].agg(' '.join, axis=1)
    name_choice = st.selectbox("Pick Name", person_data_df['Full Name'])
  else:
    st.warning("Select Atleast 1")

if by_date: #by_date return True if selected
  from_date = st.date_input("From")
  to_date = st.date_input("To")
  if(from_date > to_date):
    st.warning("Incorrect Date Interval (FROM Date Is Greater Than TO Date)")
    from_date = None
    to_date = None

if by_department: #by_department return True if selected
  dept_choice = st.selectbox("Pick Dept Name", dept_data_df.Name)
  dept_id_choice = np.where( ( (dept_data_df['Name'] == dept_choice) ), dept_data_df['ID'], None)

def find_data(passed_id_choice, passed_name_choice, passed_from_date, passed_to_date, passed_dept_choice):
  if id_choice is not None:
    id_choice = str(id_choice)
     filtered_df.query("ID == @id_choice", inplace=True) #If Inplace Is True, Changes Are Made In Current DataFrame i.e. No New Dataframe Object Is Created
      # Using @variable_name To Pass Variable In ("") Double Quotes Only Used In .query function
  if name_choice is not None:
    name_array = name_choice.split(" ")
    first_name = name_array[0]
    middle_name = name_array[1]
    last_name = name_array[2]
    #person_id = np.where( ( (person_data_df['First Name'] == first_name) & (person_data_df['Middle Name'] == middle_name) & (person_data_df['Last Name'] == last_name) ), person_data_df['ID'], person_data_df['ID'] )
    filtered_df.query("`First Name` == @first_name and `Middle Name` == @middle_name and `Last Name` == @last_name ", inplace=True)
    #filtered_df.query("`First Name` == @first_name and `Middle Name` == @middle_name and `Last Name` == @last_name ", inplace=True) # Using `` As There Is A Space In Between Column Name
  if passed_from_date is not None and passed_to_date is not None:
    formatted_from_date = passed_from_date.strftime("%d-%b-%Y") # d - Date, b - Month, Y - Year (format - yyyy)
    formatted_to_date = passed_to_date.strftime("%d-%b-%Y")
    formatted_from_date = str(formatted_from_date)
    formatted_to_date = str(formatted_to_date)
    filtered_df.query("Date > @formatted_from_date and Date < @formatted_to_date", inplace=True)
  if dept_id_choice is not None:
    dept_id_choice = str(dept_id_choice)
    filtered_df.query("Department == @dept_id_choice", inplace=True)

  return filtered_df

if submit:
  returned_filtered_df = find_data(id_choice, name_choice, from_date, to_date, dept_id_choice)
  if returned_filtered_df.empty :
    st.error("No Data Found")
  else:
    st.dataframe(returned_filtered_df)



Overwriting View.py


Running Website And Getting Authentication Token From Streamlit To Connect It To Python Code

In [ ]:
from pyngrok import ngrok
#!streamlit run View.py &
ngrok.kill()
!streamlit run View.py &>/dev/null& # IF Above Fails Then Use This
!ngrok authtoken 26I9sBkkGEDt6JCFoxECzwgoOdM_3m86LJtsg6GyEpcdZVrVC

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Getting The URL Of Website

In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect('8503')
public_url

<NgrokTunnel: "http://ff09-35-238-143-59.ngrok.io" -> "http://localhost:8503">

# Department Data

In [ ]:
%%writefile Department.py

import pandas as pd
import numpy as np
import streamlit as st
import re
person_info_path = '/content/drive/MyDrive/Data Science/Face Recognition/Person Data/person_information.csv'
department_info_path = '/content/drive/MyDrive/Data Science/Face Recognition/Department Data/department_information.csv'

person_data_df = pd.read_csv(person_info_path)
st.title("Department")
process = st.radio("",["Insert", "Update", "Delete"])

if process == "Insert":
  data = pd.DataFrame(columns=['ID', 'Name'])
  try :
    pd.read_csv(department_info_path)
  except :
    data.to_csv(department_info_path, mode='a', index=False, header=True)


  department_df = pd.read_csv(department_info_path)

  dept_id = None
  dept_name = None
  dept_id = st.number_input("ID    (100-999) ", 100,999)
  dept_name = st.text_input("Name  (2-50)", max_chars=50)
  submit_add = st.button("Add")

  if submit_add :
    if len(department_df['ID'].str.contains(f"{dept_id}")) < 1 :
      if dept_id >= 100 and dept_id <= 999 and len(dept_name) > 1 and len(dept_name) <= 50 :
        temp_data = pd.DataFrame(np.insert(data.values, len(data.index), values=[dept_id, dept_name], axis=0))
        temp_data.columns = data.columns    # Copying Columns
        data = data.append(temp_data)   # Appending Temporary Dataframe To Pre Defined Dataframe
        #Checking If Orignial Dataframe Is Empty Or Not
        if department_df.empty :    # If Empty Include Header In Orignal Dataframe i.e. Column Names
          data.to_csv(department_info_path, mode='a', index=False, header=True)
        else :
          data.to_csv(department_info_path, mode='a', index=False, header=False)
        st.success("Department Addded")
      else :
        error_str = ""
        if dept_id is None :
          error_str = error_str + "Department ID Is Empty " + " \n"
        if dept_name is "" :
          error_str = error_str + "Department Name Is Empty " + " \n"
        if dept_id > 999  or dept_id < 100 :
          error_str = error_str + "Department ID Should Be In Between (100-999) " + " \n"
        if not bool(re.findall('[0-9]+', str(dept_id) )) :
          error_str = error_str + "Department ID Only Contains Numbers " + " \n"
        if len(dept_name) < 2 or len(dept_name) > 50 :
          error_str = error_str + "Department Name Character Should Be (2-50) Characters Long " + " \n"
        if not bool(re.findall('[a-zA-Z]+', dept_name)):
          error_str = error_str + "Department Name Should Only Contain Characters " + " \n"

        st.error(error_str)
    else :
      st.error("Department ID Is Already Present In Data")


if process == "Update":
  try :
    department_df = pd.read_csv(department_info_path)
  except :  # Exception Handeling
    st.error("Department Data Is Empty")
  else : # If Exception Not Occurs
    selected_dept_id = st.selectbox("Department", department_df.ID, index=0)
    dept_name = st.text_input("Name  (3-50)", max_chars=50)
    submit_update = st.button("Update")

    # Finding Selected Department Name
    index = department_df[ department_df['ID'] == selected_dept_id ].index
    department_df.loc[index]['Name'] = dept_name
    # Storing Updated Data To File
    if not department_df.empty :
      department_df.to_csv(department_info_path, mode='w', index=False, header=True)
    st.success("Department Updated")

if process == "Delete":
  try :
    department_df = pd.read_csv(department_info_path)
  except : # Exception Handeling
    st.error("Department Data Is Empty")
  else : # If Exception Not Occurs
    rm_dept_name = st.selectbox("Department", department_df.Name, index=0)
    submit_remove = st.button("Remove")
    if submit_remove :
      #department_df['ID'] = np.where( ( (department_df['Name'] == rm_dept_name) ), department_df['ID'], None)
      # Finding Department ID From Name
      index = department_df[ department_df['Name'] == rm_dept_name ].index
      rm_dept_id = department_df.loc[index]['ID']
      # Removing Selected Department ID From Person Data
      person_data_df['Department'] = np.where( ( (person_data_df['Department'] == rm_dept_id) ), None, person_data_df['Department'])
      # Removing Selected  Department ID From Department Data
      department_df = department_df.drop(index)
      department_df.to_csv(department_info_path, mode='w', index=False, header=True)
      person_data_df.to_csv(person_info_path, mode='w', index=False, header=True)
      st.success("Department Removed")

Writing Department.py


In [ ]:
from pyngrok import ngrok
#!streamlit run Department.py &
ngrok.kill()
!streamlit run Department.py &>/dev/null& # IF Above Fails Then Use This
!ngrok authtoken 26I9sBkkGEDt6JCFoxECzwgoOdM_3m86LJtsg6GyEpcdZVrVC
public_url = ngrok.connect('8501')
public_url

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


<NgrokTunnel: "http://fe0d-35-238-143-59.ngrok.io" -> "http://localhost:80">

In [ ]:
from pyngrok import ngrok


# Final Combined  File

In [ ]:
%%writefile Combine.py
from csv import writer
from PIL import Image
import os
import re
import cv2
import datetime
import face_recognition
import pandas as pd
import numpy as np
import streamlit as st

video_path = '/content/drive/MyDrive/Data Science/Face Recognition/Videos'
image_path = '/content/drive/MyDrive/Data Science/Face Recognition/Images'
person_info_path = '/content/drive/MyDrive/Data Science/Face Recognition/Person Data/person_information.csv'
department_info_path = '/content/drive/MyDrive/Data Science/Face Recognition/Department Data/department_information.csv'

web_page = st.sidebar.selectbox("Choose your page", ["Person Data", "Department Data", "Mark Attendence", "View Attendence"])

# Person Data Webpage
if web_page == "Person Data" :
  data = pd.DataFrame(columns=['ID', 'First Name', 'Middle Name', 'Last Name', 'Gender', 'Department'])
  st.title("Person's Detail")
  process = st.radio("",["Insert", "Update", "Delete"])

  # Inserting
  if process == "Insert":
    try :
      person_data_df = pd.read_csv(person_info_path)
    except :
      data.to_csv(person_info_path, mode='a', index=False, header=True)
    try :
      dept_data_df = pd.read_csv(department_info_path)
    except : # Catching Exception
      st.error("Department Data Is Empty")
    else :
      usr_id = st.number_input("ID    (1-150) ", 1000000000,9999999999)
      usr_fname = st.text_input("First Name  (3-25)" , max_chars=25)
      usr_mname = st.text_input("Middle Name  (3-25)" , max_chars=25)
      usr_lname = st.text_input("Last Name  (3-25)" , max_chars=25)
      usr_gender = st.radio("Gender", ["Male", "Female"])
      usr_dept = st.selectbox("Department", dept_data_df.Name, index=0)
      usr_video = st.file_uploader("Upload Person's Video", type=['mp4'])
      submit = st.button("Submit")

      if submit:
        # Check If Given ID Is Already Present In DataFrame
        # If Yes Display Error Message Else Enter Data In DataFrame By Validating It
        if not usr_id in person_data_df.ID.values :
          # Validation Data Fields
          if usr_id <= 9999999999 and usr_id >=1000000000 and not bool(re.findall('[a-zA-Z]+', str(usr_id))) and len(usr_fname) > 2 and not bool(re.findall('[0-9]+', usr_fname)) and len(usr_mname) > 2 and not bool(re.findall('[0-9]+', usr_mname)) and len(usr_lname) > 2 and not bool(re.findall('[0-9]+', usr_lname)) and usr_gender != None  and usr_dept != None and usr_video != None:
            # Finding Department ID
            index = dept_data_df[ dept_data_df['Name'] == usr_dept ].index
            usr_dept_id = dept_data_df.loc[index[0], 'ID']
            #usr_dept_id = np.where( ( (dept_data_df['Name'] == usr_dept) ), dept_data_df['ID'], None)

            # Converting Person Data To Numpy Array And Storing It As Pandas Dataframe
            temp_data = pd.DataFrame(np.insert(data.values, len(data.index), values=[usr_id, usr_fname, usr_mname, usr_lname, usr_gender, usr_dept_id], axis=0))
            temp_data.columns = data.columns    # Copying Columns
            data = data.append(temp_data)   # Appending Temporary Dataframe To Pre Defined Dataframe
            # Checking If Orignial Dataframe Is Empty Or Not
            if person_data_df.empty :    # If Empty Include Header In Orignal Dataframe i.e. Column Names
              data.to_csv(person_info_path, mode='a', index=False, header=False)
            else :
              data.to_csv(person_info_path, mode='a', index=False, header=False)
            st.success("Data Entered Successfully")
            # Defining Naming Scheme For Videos Uploaded
            usr_video_name = str(usr_id)
            # Creating A Video File
            usr_video_path = os.path.join("/content/drive/MyDrive/Data Science/Face Recognition/Videos", usr_video_name)
            # Saving/Writting Video To Named File
            with open(os.path.join("/content/drive/MyDrive/Data Science/Face Recognition/Videos", usr_video_name), "wb") as f:
              f.write(usr_video.getbuffer())

            # Getting Video From Folder To Get Images
            cap = cv2.VideoCapture(video_path +"/"+ usr_video_name)
            frame = 15
            img_count = 0
            while True :
              success, image = cap.read() # Read Frame One By One Named Image
              if image is not None : # If image is There
                if frame % 15 == 0 and img_count < 10 : # Take Every 15th Frame And If Count Of Image Is Less Than 10
                  img_count = img_count + 1
                  if(img_count < 10): # If Image Count is Less Than 10 Then Add 0 As A Prefix In Image Count
                    save_img_name = usr_video_name + "_" + "0" + str(img_count) + ".jpg"
                    cv2.imwrite('/content/drive/MyDrive/Data Science/Face Recognition/Images/'+save_img_name , image) # Writting Image In Folder
                  else :
                    save_img_name = usr_video_name + "_" + str(img_count) + ".jpg"
                    cv2.imwrite('/content/drive/MyDrive/Data Science/Face Recognition/Images/'+save_img_name , image)
                frame = frame + 1 # Increasing Frame Counter
              else :
                break
            st.success("Video Uploaded Successfully")
          else :
            error_str = ""
            if usr_id >= 9999999999 and usr_id <=1000000000 :
              error_str = error_str + "Enter Proper ID"
            if len(usr_fname) > 25 or len(usr_fname) < 2 :
              error_str = error_str + "First Name Length Should Be (3-25) Characters " + "\n"
            if not bool(re.findall('[0-9]+', usr_fname)) :
              error_str = error_str + "First Name Should Contain Only Alphabets " + "\n"
            if len(usr_mname) < 25 or len(usr_mname) < 2 :
              error_str = error_str + "Middle Name Length Should Be (3-25) Characters " + "\n"
            if not bool(re.findall('[0-9]+', usr_mname)) :
              error_str = error_str + "Middle Name Should Contain Only Alphabets " + "\n"
            if len(usr_lname) > 25 or len(usr_lname) < 2 :
              error_str = error_str + "Length Should Be (3-25) Characters " + "\n"
            if not bool(re.findall('[0-9]+', usr_lname)) :
              error_str = error_str + "Last Name Should Contain Only Alphabets " + "\n"
            if usr_gender is None:
              error_str = error_str + "Gender Is Not Selected " + "\n"
            if usr_dept is None:
              error_str = error_str + "Department Is Not Selected " + "\n"
            if usr_video is None:
              error_str = error_str + "Video File Is Empty " + "\n"
            st.error(error_str)
        else :
          st.error("Department ID Is Already Present In Data")

  # Updating
  if process == "Update":
    try :
      person_data_df = pd.read_csv(person_info_path)
    except : # Catching Exception
      st.error("Person Data Is Empty")
    else : # If Exception Not Occurs
      try :
        dept_data_df = pd.read_csv(department_info_path)
      except :
        st.error("Department Data is Empty")
      else :
        re_video = None
        selected_id = st.selectbox("ID", person_data_df.ID, index=0)

        # Finding Person Details  From Selected ID
        person_index = person_data_df[ person_data_df['ID'] == selected_id ].index[0]
        usr_fname = person_data_df.loc[person_index]["First Name"]
        usr_mname = person_data_df.loc[person_index]["Middle Name"]
        usr_lname = person_data_df.loc[person_index]["Last Name"]
        usr_gender = person_data_df.loc[person_index]["Gender"]
        usr_dept_id = person_data_df.loc[person_index]["Department"]
        if str(usr_dept_id) == 'nan' :
          usr_dept_id = 100
        temp_dept_index = dept_data_df[ dept_data_df['ID'] == str(int(usr_dept_id)) ].index[0]
        if temp_dept_index == None :
          dept_index = 1
        else :
          dept_index = temp_dept_index
        #temp_dept_index = dept_data_df[ dept_data_df['ID'] == usr_dept_id ].index
        #st.write(dept_data_df[ dept_data_df['ID'] == usr_dept_id ].index)
        # Assigning Values To Load Data In User Inputs
        usr_fname = st.text_input("First Name  (3-25)" , max_chars=25, value = str(usr_fname))
        usr_mname = st.text_input("Middle Name  (3-25)" , max_chars=25, value = str(usr_mname))
        usr_lname = st.text_input("Last Name  (3-25)" , max_chars=25, value = str(usr_lname))
        gen_index = 0
        if str(usr_gender[0]) == "Male" :
          gen_index = 0
        elif str(usr_gender[0]) == "Female" :
          gen_index = 1
        usr_gender = st.radio("Gender", ["Male", "Female"], index=gen_index)
        usr_dept = st.selectbox("Department", dept_data_df.Name, index= int(dept_index) )
        re_video = st.checkbox("Re-Upload Video")
        if re_video:
          usr_video = st.file_uploader("Upload Person's Video", type=['mp4'])

        update = st.button("Update")

        if update:
          # Validation Data Fields
          if len(usr_fname) > 2 and not bool(re.findall('[0-9]+', usr_fname)) and len(usr_mname) > 2 and not bool(re.findall('[0-9]+', usr_mname)) and len(usr_lname) > 2 and not bool(re.findall('[0-9]+', usr_lname)) and usr_gender != None and usr_dept != None :
            # Finding Department ID
            usr_dept_id_index = dept_data_df[ dept_data_df['Name'] == usr_dept ].index
            usr_dept_id = dept_data_df.loc[usr_dept_id_index[0], "ID"]
            # Updating Data In Dataframe
            index = person_data_df[ person_data_df['ID'] == selected_id ].index
            person_data_df.loc[index, "First Name"] = usr_fname
            person_data_df.loc[index, "Middle Name"] = usr_mname
            person_data_df.loc[index, "Last Name"] = usr_lname
            person_data_df.loc[index, "Gender"] = usr_gender
            person_data_df.loc[index, "Department"] = int(usr_dept_id)
            # Saving Data In Dataframe
            person_data_df.to_csv(person_info_path, mode='w', index=False, header=True)
            st.write(pd.DataFrame(person_data_df))
            st.success("Data Updated Successfully")
            if re_video and usr_video is not None :
              # Defining Naming Scheme For Videos Uploaded
              usr_video_name = str(usr_id)
              # Creating A Video File
              usr_video_path = os.path.join("/content/drive/MyDrive/Data Science/Face Recognition/Videos", usr_video_name)
              # Saving/Writting Video To Named File
              with open(os.path.join("/content/drive/MyDrive/Data Science/Face Recognition/Videos", usr_video_name), "wb") as f:
                f.write(usr_video.getbuffer())

              # Getting Video From Folder To Get Images
              cap = cv2.VideoCapture(video_path +"/"+ usr_video_name)
              frame = 15
              img_count = 0
              while True :
                success, image = cap.read() # Read Frame One By One Named Image
                if image is not None : # If image is There
                  if frame % 15 == 0 and img_count < 10 : # Take Every 15th Frame And If Count Of Image Is Less Than 10
                    img_count = img_count + 1
                    if(img_count < 10): # If Image Count is Less Than 10 Then Add 0 As A Prefix In Image Count
                      save_img_name = usr_video_name + "_" + "0" + str(img_count) + ".jpg"
                      cv2.imwrite('/content/drive/MyDrive/Data Science/Face Recognition/Images/'+save_img_name , image) # Writting Image In Folder
                    else :
                      save_img_name = usr_video_name + "_" + str(img_count) + ".jpg"
                      cv2.imwrite('/content/drive/MyDrive/Data Science/Face Recognition/Images/'+save_img_name , image)
                  frame = frame + 1 # Increasing Frame Counter
                else :
                  break
              st.write("Video Uploaded Successfully")
          else :
            error_str = ""
            if len(usr_fname) < 25 and len(usr_fname) > 2 :
              error_str = error_str + "First Name Length Should Be (3-25) Characters " + "\n"
            if not bool(re.findall('[0-9]+', usr_fname)) :
              error_str = error_str + "First Name Should Contain Only Alphabets " + "\n"
            if len(usr_mname) < 25 and len(usr_mname) > 2 :
              error_str = error_str + "Middle Name Length Should Be (3-25) Characters " + "\n"
            if not bool(re.findall('[0-9]+', usr_mname)) :
              error_str = error_str + "Middle Name Should Contain Only Alphabets " + "\n"
            if len(usr_lname) < 25 and len(usr_lname) > 2 :
              error_str = error_str + "Length Should Be (3-25) Characters " + "\n"
            if not bool(re.findall('[0-9]+', usr_lname)) :
              error_str = error_str + "Last Name Should Contain Only Alphabets " + "\n"
            if usr_gender is None:
              error_str = error_str + "Gender Is Not Selected " + "\n"
            if usr_dept is None:
              error_str = error_str + "Department Is Not Selected " + "\n"
            if re_video is not None and usr_video is None:
              error_str = error_str + "Video File Is Empty " + "\n"
            st.error(error_str)

  # Deleting
  if process == "Delete":
    try:
      person_data_df = pd.read_csv(person_info_path)
    except:
      st.error("Person Data Is Empty")
    else:
      selected_id = st.selectbox("ID", person_data_df.ID, index=0)
      delete = st.button("Delete")
      # To Display The Data of Selected ID
      selected_id_df = person_data_df.query("ID == @selected_id", inplace=False)
      if not selected_id_df.empty:
        st.dataframe(selected_id_df)

      if delete:
        # Dropping / Deleting Person By Finding Index Of ID In File
        # ( person_data_df.query("ID == @selected_id").index Is Similar ) To ( person_data_df[ person_data_df['ID'] == selected_id].index )
        new_person_data_df = person_data_df.drop(person_data_df.query("ID == @selected_id").index)
        # Saving Data
        new_person_data_df.to_csv(person_info_path, mode='w', index=False, header=True)
        err_str = ""
        # Removing Person's Video File
        try :
          os.remove(f'{video_path}/{selected_id}')
        except :
          err_str = err_str + " Video Already Deleted " + "\n"
        # Removing Person's Images
        img_counter = 1
        err_str = ""
        while img_counter <= 10 :
          if img_counter % 10 != 0 :
            try :
              os.remove(f'{image_path}/{person}_0{img_counter}.jpg')
            except :
              err_str = err_str + " Images Already Deleted"
          elif img_counter % 10 == 0 :
            try :
              os.remove(f'{image_path}/{person}_{img_counter}.jpg')
            except :
              err_str = err_str + " Images Already Deleted"
          img_counter = img_counter + 1

        if err_str != "" :
          print(err_str)

        if new_person_data_df.query("ID == @selected_id").empty :
          st.success("Data Deleted")
        else :
          st.error("Error Occured When Deleting Data")

# Department Data Webpage
if web_page == "Department Data":
  global person_flag
  person_flag = 1 # Data In Person Information Is Present
  try :
    person_data_df = pd.read_csv(person_info_path)
  except :
    person_flag = 0 # Data In Person Information Is Absent

  st.title("Department")
  process = st.radio("",["Insert", "Update", "Delete"])

  if process == "Insert":
    data = pd.DataFrame(columns=['ID', 'Name'])
    try :
      pd.read_csv(department_info_path)
    except :
      data.to_csv(department_info_path, mode='a', index=False, header=True)

    department_df = pd.read_csv(department_info_path)

    dept_id = None
    dept_name = None
    dept_id = st.number_input("ID    (100-999) ", 100,999)
    dept_name = st.text_input("Name  (2-50)", max_chars=50)
    submit_add = st.button("Add")

    if submit_add :

      # Check If Given ID Is Not In Dataframe Then Add
      if not str(dept_id) in department_df.ID.values :
        if dept_id >= 100 and dept_id <= 999 and len(dept_name) > 1 and len(dept_name) <= 50 and not bool(re.findall('[0-9]+', dept_name)) :
          temp_data = pd.DataFrame(np.insert(data.values, len(data.index), values=[dept_id, dept_name], axis=0))
          temp_data.columns = data.columns    # Copying Columns
          data = data.append(temp_data)   # Appending Temporary Dataframe To Pre Defined Dataframe
          #Checking If Orignial Dataframe Is Empty Or Not
          if department_df.empty :    # If Empty Include Header In Orignal Dataframe i.e. Column Names
            data.to_csv(department_info_path, mode='a', index=False, header=False)
          else :
            data.to_csv(department_info_path, mode='a', index=False, header=False)
          st.success("Department Addded")
        else :
          error_str = ""
          if dept_id is None :
            error_str = error_str + "Department ID Is Empty " + " \n"
          if dept_name is "" :
            error_str = error_str + "Department Name Is Empty " + " \n"
          if dept_id > 999  or dept_id < 100 :
            error_str = error_str + "Department ID Should Be In Between (100-999) " + " \n"
          if not bool(re.findall('[a-zA-Z]+', str(dept_id) )) :
            error_str = error_str + "Department ID Only Contains Numbers " + " \n"
          if len(dept_name) < 2 or len(dept_name) > 50 :
            error_str = error_str + "Department Name Character Should Be (2-50) Characters Long " + " \n"
          if not bool(re.findall('[0-9]+', dept_name)):
            error_str = error_str + "Department Name Should Only Contain Characters " + " \n"

          st.error(error_str)
      else :
        st.error("Department ID Is Already Present In Data")

  if process == "Update":
    try :
      department_df = pd.read_csv(department_info_path)
    except :  # Exception Handeling
      st.error("Department Data Is Empty")
    else : # If Exception Not Occurs
      selected_dept_id = st.selectbox("Department", department_df.ID, index=1)
      dept_index = department_df[ department_df['ID'] == selected_dept_id ].index[0]
      dept_name = department_df.loc[dept_index]['Name']
      dept_name = st.text_input("Name  (3-50)", max_chars=50, value = dept_name)

      submit_update = st.button("Update")
      if submit_update :
        if selected_dept_id == "ID" :
          st.error("Select Proper ID")
        else :
          error_str = ""
          if dept_name is "" :
            error_str = error_str + "Department Name Is Empty " + " \n"
          if len(dept_name) < 2 or len(dept_name) > 50 :
            error_str = error_str + "Department Name Character Should Be (2-50) Characters Long " + " \n"
          if not bool(re.findall('[a-zA-Z]+', dept_name)):
            error_str = error_str + "Department Name Should Only Contain Characters " + " \n"
          if error_str != "":
            st.error(error_str)
          else :
            # Finding Selected Department Name And Updating
            index = department_df[ department_df['ID'] == selected_dept_id ].index[0]
            department_df.loc[index]['Name'] = dept_name

            # Storing Updated Data To File
            if not department_df.empty :
              department_df.to_csv(department_info_path, mode='w', index=False, header=True)
              st.success("Department Updated")

  if process == "Delete":
    try :
      department_df = pd.read_csv(department_info_path)
    except : # Exception Handeling
      st.error("Department Data Is Empty")
    else : # If Exception Not Occurs
      rm_dept_name = st.selectbox("Department", department_df.Name, index=0)
      submit_remove = st.button("Remove")
      if submit_remove :
        if rm_dept_name == "Name" :
          st.error("Select Proper Department Name")
        else :
          # Finding Department ID From Name
          index = department_df[ department_df['Name'] == rm_dept_name ].index[0]
          rm_dept_id = department_df.loc[index]['ID']
          if person_flag == 1 :
            # Removing Selected Department ID From Person Data
            person_data_df['Department'] = np.where( ( (person_data_df['Department'] == rm_dept_id) ), None, person_data_df['Department'])

          # Removing Selected  Department ID From Department Data
          department_df = department_df.drop(index) # By Default axis = 0 i.e. Rows In DataFrame.drop()
          department_df.to_csv(department_info_path, mode='w', index=False, header=True)
          person_data_df.to_csv(person_info_path, mode='w', index=False, header=True)
          st.success("Department Removed")

# Attendence Marking Webpage
if web_page == "Mark Attendence":
  attendance_path = '/content/drive/MyDrive/Data Science/Face Recognition/attendance.csv'

  data = pd.DataFrame(columns=['ID', 'Date', 'Entry Time', 'Exit Time'])
  try :
    person_info_df = pd.read_csv(person_info_path)
  except :
    st.error("Person Data Is Empty")
  else :
    try :
      attendance_df = pd.read_csv(attendance_path)
    except :
      data.to_csv(attendance_path, mode='a', index=False, header=True)
    else :
      # Function To Find The Encoded Data Of The Training Image
      def findEncodings(images):
        encodeList = []
        for img in images:
          img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
          #encode = face_recognition.face_encodings(img)[0]
          encode = face_recognition.face_encodings(img)
          if len(encode) > 0 :
            encodeList.append(encode[0])
          #encodeList.append(encode)
        return encodeList

      # Function To Recognise Person And Return ID And Name
      def recognize_face(passed_image):
        path = '/content/drive/MyDrive/Data Science/Face Recognition/Images'
        images = []
        classNames = []
        #We define two empty lists for storing training images and the classNames, which means the person’s name.
        # Reading the training images and classes and storing into the corresponding lists
        for img in os.listdir(path):
          image = cv2.imread(f'{path}/{img}')
          images.append(image)
          classNames.append(os.path.splitext(img)[0])
        # Find Encodings Of Trained Images
        knownEncodes = findEncodings(images)
        #Note – Change this scale according to your need between 0 and 1.
        # A lower number will give better performance but it will not be able to detect faces if the face is small in the image,
        # and a greater number can detect small faces in the image but the performance will be slow
        scale = 0.25

        Current_image = cv2.resize(passed_image,(0,0),None,scale,scale)
        Current_image = cv2.cvtColor(Current_image, cv2.COLOR_BGR2RGB)
        # Find the face location and encodings for the current frame
        face_locations = face_recognition.face_locations(Current_image,  model='cnn')
        face_encodes = face_recognition.face_encodings(Current_image,face_locations)
        name = ''
        for encodeFace,faceLocation in zip(face_encodes,face_locations):
          matches = face_recognition.compare_faces(knownEncodes,encodeFace, tolerance=0.6)
          faceDis = face_recognition.face_distance(knownEncodes,encodeFace)
          matchIndex = np.argmin(faceDis)
          # If match found then get the class name for the corresponding match
          if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            return name
          else:
            name = 'Unknown'
        return name

      # Function To Mark The Attendance Of The Person
      def mark_attendance(passed_id) :
        data = pd.DataFrame(columns=['ID', 'Date', 'Entry Time', 'Exit Time'])
        #person_info_df_modified = person_info_df.set_index("ID", inplace=False)
        #person_dept =  person_info_df_modified.loc[passed_id]['Department']

        datetime_obj = datetime.datetime.now() # Creating Date Time Object
        current_date = datetime_obj.strftime("%d-%b-%Y") # Getting Current Date
        current_time = datetime_obj.strftime("%H:%M:%S") # Getting Current Time

        attendance_df_modified = attendance_df.copy(deep=True) # Deep = True, Modifications To The Data Or Indices Of The Copy Will Not Be Reflected In The Original Object
        #attendance_df_modified = attendance_df.set_index("ID", inplace=False)

        # passed_id is already in string fromat
        str_passed_id = str(passed_id)
        str_current_date = str(current_date)
        if attendance_df.loc[ (attendance_df['ID'] == int(passed_id)) & (attendance_df['Date'] == str(current_date)) ].index.empty :
          entry_time = current_time
          exit_time = None # None Will Convert Into NaN In Dataframe
          # Code for marking attendance Entry Time
          temp_data = pd.DataFrame(np.insert(data.values, len(data.index), values=[int(passed_id), str(current_date), str(entry_time), str(exit_time)], axis=0))
          temp_data.columns = data.columns    # Copying Columns
          data = data.append(temp_data)   # Appending Temporary Dataframe To Pre Defined Dataframe

          # Saving Data To DataFrame
          data.to_csv(attendance_path, mode='a', index=False, header=False)
          return True
        elif not attendance_df.loc[ (attendance_df['ID'] == int(passed_id)) & (attendance_df['Date'] == str(current_date)) ].index.empty :
          # Code for marking attendance Exit Time
          exit_time = current_time
          integer_passed_id = int(passed_id)
          index = attendance_df.loc[ (attendance_df['ID'] == int(passed_id)) & (attendance_df['Date'] == str(current_date)) ].index[0] # index[0] As Will Return Value Instead Of Value As Well As Information
          attendance_df.loc[index, 'Exit Time'] = exit_time
          #attendance_df["Exit Time"] = np.where( ( (attendance_df['ID'] == passed_id) & (attendance_df['Date'] == current_date) ), exit_time, attendance_df["Exit Time"])
          # Saving Data To DataFrame
          attendance_df.to_csv(attendance_path, mode='w', index=False, header=True)
          return True
        else :
          #st.error("Error Occured Try Again")
          return False

      st.title("Mark Your Attendence")
      input_image = st.camera_input("Take Your Picture")

      # USING SPINNER STATUS ELEMENT TO REPRESENT LOADING WIDGET
      with st.spinner():
        if input_image is not None:
          # Converting Streamlit Image To OpenCV Image
          image_byte_data = input_image.getvalue() # Getting The Byte Value Of Image
          cv_image = cv2.imdecode(np.frombuffer(image_byte_data, np.uint8), cv2.IMREAD_COLOR) # Converting Byte Image OpenCV Image
          cv_image_rgb = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB) # Converting Image To RGB Color Scheme

          person = recognize_face(cv_image_rgb) # Passing The Image To Function To Get ID And Name Of Person
          if person == "Unknown" or person is None or person == "" : # If Person Is Not Recognized Then Display Error Message
            st.error("Error Person Not Found, Retry")
          else : # If Person Is Recognized Then Get ID From The Lable Of Matched Image
            person_array = person.split("_")    # Splitting Lable To Split Image Name By Underscore(_)
            person_id = person_array[0] # Getting The First Object In Array As It Contains Person ID
            # Passing ID To Mark The Attendance Of Recognised Person
            if mark_attendance(person_id):
              # Display Success Message With Person ID
              st.success("Attendance Marked Person : "+ person_id)
            else :
              # Displaying Error Message If Error Occurs
              st.error("Error Occured, Try Again")

# Attendence Viewing Webpage
if web_page == "View Attendence":
  try :
   person_data_df = pd.read_csv(person_info_path)
  except : # Catching Exception
    st.error("Person Data Is Empty")
  try :
    dept_data_df = pd.read_csv(department_info_path)
  except :
    st.error("Department Data Is Empty")
  try :
    attendance_df = pd.read_csv(attendance_path)
  except :
    st.error("Attendence Data Is Empty")
  else :
    filtered_df = attendance_df.copy(deep=True) # True if Changes Should Not Reflect In Original DataFrame
    filtered_df = filtered_df.merge(person_data_df,on='ID',how="left") # Merging / Left Join On Attendance Table And Person Table By ID
    filtered_df.drop(['Gender'], axis=1) #Dropping Gender Column,  axis=1 As We Are Dropping Column
    filtered_df = filtered_df.loc[:,~filtered_df.columns.duplicated()].copy() # [ROW, COLUMN] : Is Used As We Need All Rows And Non-Duplicate Columns i.e. ID Column
    # ~ Is Used To Invert Answer i.e. If Column Is Duplicate Answer Will Be True And Only That Column Will Be Selected
    filtered_df = filtered_df[[ 'ID', 'First Name', 'Middle Name', 'Last Name', 'Date', 'Entry Time', 'Exit Time', 'Department' ]]

    #person_data_df = person_data_df.set_index("ID", inplace=False) #If True, modifies the DataFrame in place (do not create a new object).

    name_or_id = None
    name_choice = None
    id_choice = None
    from_date = None
    to_date = None
    dept_choice = None
    dept_id_choice = None

    st.title("View Attendance")
    by_id_name = st.checkbox("Filter By ID / Name")
    by_date = st.checkbox("Filter By Date Interval")
    by_department = st.checkbox("Filter By Department")
    submit = st.button("Submit")

    if by_id_name: #by_id_name return True if selected
      name_or_id = st.radio("Select Filter Criteria", ["ID", "Name"])
      if name_or_id == "ID":
        id_choice = st.selectbox("Pick ID", person_data_df.ID)
      elif name_or_id == "Name":
        # Combining FName, MName, LName And Creating New Column Full Name
        person_data_df['Full Name'] = person_data_df[['First Name', 'Middle Name', 'Last Name']].agg(' '.join, axis=1)
        name_choice = st.selectbox("Pick Name", person_data_df['Full Name'])
      else:
        st.warning("Select Atleast 1")

    if by_date: #by_date return True if selected
      from_date = st.date_input("From")
      to_date = st.date_input("To")
      if(from_date > to_date):
        st.warning("Incorrect Date Interval (FROM Date Is Greater Than TO Date)")
        from_date = None
        to_date = None

    if by_department: #by_department return True if selected
      dept_choice = st.selectbox("Pick Dept Name", dept_data_df.Name)
      dept_id_choice = np.where( ( (dept_data_df['Name'] == dept_choice) ), dept_data_df['ID'], None)

    def find_data(passed_id_choice, passed_name_choice, passed_from_date, passed_to_date, passed_dept_choice):
      if id_choice is not None and not filtered_df.empty :
        id_choice = str(id_choice)
        filtered_df.query("ID == @id_choice", inplace=True) #If Inplace Is True, Changes Are Made In Current DataFrame i.e. No New Dataframe Object Is Created
        # Using @variable_name To Pass Variable In ("") Double Quotes Only Used In .query function
      if name_choice is not None and not filtered_df.empty :
        name_array = name_choice.split(" ")
        first_name = name_array[0]
        middle_name = name_array[1]
        last_name = name_array[2]
        #person_id = np.where( ( (person_data_df['First Name'] == first_name) & (person_data_df['Middle Name'] == middle_name) & (person_data_df['Last Name'] == last_name) ), person_data_df['ID'], person_data_df['ID'] )
        filtered_df.query("`First Name` == @first_name and `Middle Name` == @middle_name and `Last Name` == @last_name ", inplace=True)
        #filtered_df.query("`First Name` == @first_name and `Middle Name` == @middle_name and `Last Name` == @last_name ", inplace=True) # Using `` As There Is A Space In Between Column Name
      if passed_from_date is not None and passed_to_date is not None and not filtered_df.empty:
        formatted_from_date = passed_from_date.strftime("%d-%b-%Y") # d - Date, b - Month, Y - Year (format - yyyy)
        formatted_to_date = passed_to_date.strftime("%d-%b-%Y")
        formatted_from_date = str(formatted_from_date)
        formatted_to_date = str(formatted_to_date)
        filtered_df.query("Date > @formatted_from_date and Date < @formatted_to_date", inplace=True)
      if dept_id_choice is not None and not filtered_df.empty:
        dept_id_choice = str(dept_id_choice)
        filtered_df.query("Department == @dept_id_choice", inplace=True)

      return filtered_df

    if submit:
      returned_filtered_df = find_data(id_choice, name_choice, from_date, to_date, dept_id_choice)
      if returned_filtered_df.empty :
        st.error("No Data Found")
      else:
        st.dataframe(returned_filtered_df)

Writing Combine.py


In [ ]:
from pyngrok import ngrok
#!streamlit run Department.py &
ngrok.kill()
!streamlit run Combine.py &>/dev/null& # IF Above Fails Then Use This
!ngrok authtoken YOUR_NGROK_AUTHENTICATION_TOKEN
public_url = ngrok.connect('8501')
public_url

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


<NgrokTunnel: "https://1ee3-35-185-189-169.ngrok-free.app" -> "http://localhost:8501">